# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
import os
# from utils import get_openai_api_key, get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()
# Setup the Gemini pro LLM
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0, verbose=True, google_api_key=os.environ["GOOGLE_API_KEY"],convert_system_message_to_human=True)

#setup the serper api key
os.environ["SERPER_API_KEY"] = os.getenv("serper")

## crewAI Tools

In [5]:
with open('inputs\Ram_Resume_2024.pdf', 'r', encoding='latin-1') as infile:
    content = infile.read()

In [6]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  PDFSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='inputs\Ram_Resume_2024.pdf')
semantic_search_resume = PDFSearchTool(mdx=r"inputs\Ram_Resume_2024.pdf")

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [7]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))
from IPython.display import IFrame
IFrame('inputs\Ram_Resume_2024.pdf', width=600, height=400)

## Creating Agents

In [8]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm = llm
)

In [9]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = llm
)

In [10]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm = llm
)

In [11]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = llm
)

## Creating Tasks

In [12]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [13]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [14]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [15]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [16]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [17]:
job_application_inputs = {
    'job_posting_url': 'https://www.google.com/search?q=IBM+Generative+AI+developer+job&oq=IBM+Generative+AI+developer+job&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIGCAEQLhhA0gEJMTM0NzFqMGoxqAIAsAIA&sourceid=chrome&ie=UTF-8&ibp=htl;jobs&htidocid=QW-A1gKy_AIkmQGNAAAAAA%3D%3D&sa=X&ved=2ahUKEwik3qaz57WGAxXmk1YBHbjXAHkQkd0GegQIERAB#fpstate=tldetail&htivrt=jobs&htiq=IBM+Generative+AI+developer+job&htidocid=QW-A1gKy_AIkmQGNAAAAAA%3D%3D&sxsrf=ADLYWIIMiqTXv_35TSLVUbF9aCG8-yLomg:1717086961911',
    'github_url': 'https://github.com/Ram580',
    'personal_writeup': """Highly motivated and results-oriented Data Analyst with 2.5 years of experience driving data-driven decision making through advanced analytics, ML, Deep learning, NLP and Generative AI solutions.
    adept at various domains – marketing analytics , manufacturing, pharma and airlines.  and expert in multiple
    programming languages and frameworks.His Experties are Advanced Analytics : Utilized Machine learning(Regression, Classification, Time series forecasting), Deep Learning (LSTM), NLP, and GenAI (Langchain) to solve complex business problems in Pharma and Healthcare.
ETL Pipeline Development: Built and automated efficient ETL pipelines  leveraging Python, SQL, PySpark that reduced data turnaround time by 30%.
Generative AI: Expertise in LLM fine-tuning (PEFT), RLHF, prompt engineering for various AI models, RAG Applications, Multimodal AI applications development, evaluation and deployment,
Actionable Insights & Dashboards: Created high-performance dashboards (Power BI, Tableau) to empower data-driven decision making (e.g., patient churn reduction).
Forecasting & Modeling: Built, finetuned and enhanced multiple forecasting models for various use cases like regression, classification, time series forecasting etc.,
Project Leadership: Mentored data science interns and led junior analysts in various data science projects (EDA to deployment).
Cross-Industry Experience: Possesses experience in diverse sectors (Pharma, Healthcare, Airlines).
Project Versatility: Implemented projects ranging from prescriber choice model for measuring campaign effectiveness in pharma, airline resource allocation (passenger footfall forecasting), Gen AI knowledge repositories chatbot (Langchain, LLMs).
Cloud and Deployment Skills: Utilized tools like Docker, Git, Github Actions and AWS Sagemaker, Amazon Bedrock, S3, AWS Lambda, API Gateway, AWS RDS for project deployments
"""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [18]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

RAM BADANA
Data Scientist
Email: ramhemanth580@gmail.com
Mobile: +91-6303774781
LinkedIn: @Ram Badana
Hugging Face: Ram HF profile
GitHub: Ram580

Summary
Highly motivated and results-oriented Data Analyst with 2.5 years of experience driving data-driven decision-making through advanced analytics, ML, Deep learning, NLP, and Generative AI solutions. Proven ability to design, develop, and deploy effective ETL pipelines, build and operationalize predictive models, and translate complex data insights into actionable strategies. Proficient in optimizing processes and leveraging advanced ML techniques to consistently deliver impactful results. Adept at various domains – (Pharma, Healthcare, Airlines), quickly adaptable to new technologies.

Skills
- Programming Language & Packages: Python (Tensorflow, keras, Pytorch, Transformers, Sklearn, NLTK), SQL, PySpark
- Data Science: Machine Learning (Regression, Classification, Clustering), Deep Learning (ANN, CNN, Sequence models)
- Generative AI: LLMs, Finetuning using PEFT Techniques (LORA, QLORA), RLHF, Multimodal RAG applications, Multi-Agent AI system, RAG Evaluation Techniques (RAGAS), Vector DBs, NoSQL DBs, Langchain – (chains, Tools, Agents), Llama Index, LLMOPS, Hugging Face
- Technology & Cloud Platform: AWS, Amazon Bedrock, AWS Sagemaker, Data bricks(beginner), Docker, Github Actions, FastAPI, Flask Rest API, Git

Education
Manipal University 2018 – 2022
B. Tech Automotive Engineering (Minor specialization in EV and Hybrid vehicle Technology)
- CGPA: 8.2

Accomplishments
- 2 X Ambition and Hunger Award – TheMathCompany.
- 3 X Appreciation Awards from Multiple Project clients at Mathco.
- IBM certified Data scientist.
- Secured 2nd place in Mathco Hackathon

Certifications
- IBM Data science professional Certification.
- Deep learning Specialization by Deeplearning.AI.
- AWS Generative AI Using LLMs Certification.
- Azure Power BI developer.

Experience
Data Analyst - TheMathCompany, Bengaluru, India
June 2022 – Current
- Developed a comprehensive Power BI dashboard suite leveraging Power BI, SQL,

DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

Education
MBA in Information Technology
London Business School - MBA

Advanced Leadership Techniques
University of London - Certification

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

Based on the resume and job requirements, here are some potential interview questions and talking points for the candidate:
1. Can you elaborate on your experience with Generative AI, particularly in relation to LLMs and other techniques mentioned in your resume?
2. How have you collaborated with engineers and scientists to design and develop innovative AI solutions in your previous roles?
3. Could you provide an example of a project where you led the implementation of an AI solution and the challenges you faced?
4. How do you stay updated with the latest AI technologies and trends, and how do you apply them in your work?
5. Describe a situation where you had to troubleshoot technical issues related to AI products and solutions. How did you approach and resolve the problem?
6. How do you ensure the performance of AI products and solutions is monitored effectively and improvements are recommended?
7. Can you discuss your experience in developing technical documentation, such as user manuals and training materials, for AI solutions?
8. How have you leveraged your skills in Machine Learning, Deep Learning, and Generative AI to optimize processes and deliver impactful results in your previous projects?
9. What experience do you have with cloud platforms like AWS and technologies like Docker and GitHub Actions in the context of AI development?
10. How do you approach working in a team environment, especially when collaborating with engineers and scientists from diverse backgrounds?

These questions and talking points can help the candidate showcase their expertise in Generative AI and demonstrate how their skills align with the job requirements.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)